# CHEM INTRODUCTION

### What is CHEM and PyCHEM

CHEM stands for channel emulator. We've modified the hardware drivers to communicate over network to transfer signals instead of through hardware. Channel Emulator takes the raw I/Q samples and applies channels effects(Path Loss, Ch. Coeffs., etc.). Channel Emulator runs inside the CHEM-VM which experimenters don't have access to it. To configure CHEM for different scenarios, it needs to be controlled with pyCHEM. pyCHEM is a python library for communicating with CHEM remotely. 

### How To Connect CHEM with pyCHEM

There are 2 options,

* __Jupyter Notebook__
    
    Jupyter notebook is python notebook interface which allows live coding in Python. It runs as a server and user can connect to it with the web interface Jupyter provides. User will be able to execute code inside the OEO-CONSOLE without trying to run it over command line. 
    
    To connect,
    * ssh to OEO-CONSOLE with port forwarding to 8888 
    ```
     ~ ssh -L localhost:8888:localhost:8888 root@<oeo-console-ip-address>
    ```
    * After establishing ssh connection to OEO-Console, go to the directory you want to work on(for saving your project files)
    * Jupyter notebook should be started with the command below
    ```
     ~ python3 -m notebook
    ```
    * Previous command will start the Jupyter notebook web server and provide link similar to output below,
    <div class="alert alert-block alert-warning">
        To access the notebook, open this file in a browser:<br>                
        <a href="">http://localhost:8888/?token=2d4e9f5b1ea0fa4031fe4bd287303db89400942f1d714cd4</a> <br>
     or <a href="">http://127.0.0.1:8888/?token=2d4e9f5b1ea0fa4031fe4bd287303db89400942f1d714cd4</a>
    </div>
    <br>
    * After opening the localhost link that is provided in the command line, you will be able to use PyCHEM
* __Python Shell__

    Python shell is much easier to access but it might be hard to use in the long term. You will be using command line interface of Python. User should keep this in mind that nothing is saved on Python shell. It mostly useful quick tests.
    * User can connect OEO-CONSOLE without port forwarding such as below 
    ```
     ~ ssh root@<oeo-console-ip-address>
    ```
    * After establishing ssh connection, you just need to run the command below,
    ```
     ~ python3 
    ```
    

After accessing to any of the Python interfaces above, you can send commands to CHEM with pyCHEM.


#  Setup of pyCHEM Environment and Getting Information from CHEM

In [1]:
## First import the pychem with the command below
from pychem import *

The below code provides the connection to CHEM. You are not going to provide the ip address in the OEO-CONSOLE. The code below is for demonstration purposes. It creates an object called ```ch``` and user needs to use it for all future changes in the CHEM.

In [7]:
ch = CHEM()

To get the nodes that are connected to CHEM you can run the command below. It will return a boolean. You can access nodes from ```ch.nodes``` This will allow you to see information related to nodes(frequencies, sampling rates, number of channels, etc.)

In [8]:
ch.getNodes()

None


True

In [4]:
ch.nodes["enb-1"]["rxsampleRate"]

TypeError: 'NoneType' object is not subscriptable

To get the recent channel information, the ``` getChannels ``` function needs to be called. It will return a boolean regarding the process. If it successfully received all the information, it will return True. After successfull retrieval of channel information, channel dictionary can be accessed with ``` ch.channels ``` variable. The channel dictionary has the information as shown below,

```
{
    channel-id(freq in MHz) : {
         pathLoss: pathLossModel,
         sRate: samplingRate
    }
}

```


In [6]:
ch.getChannels()

None


True

The above function only provides information related to frequency channels. User can only modify __path loss__ from that information. It might be confusing but there are more channels when we think about individual paths between nodes. Therefore, ``` getIndividualChannels``` function exist. It parses individual channel information from CHEM. The information provides  
```
{
    freq: {
       individual-channel-id: channel-coefficient
    }
}

```

The ``` individual-channel-id``` is the key for changing the channel coefficients. Therefore, it is the primary key to point out to individual channel.

In [6]:
ch.getIndividualChannels()

None


True

In [7]:
{'2560.000000': {'enb-1-CH-0==>ue-1-CH-0': 1.0}, '2680.000000': {'ue-1-CH-0==>enb-1-CH-0': 1.0, 'ue-1-CH-0==>enb-1-CH-1': 1.0}}

{'2560.000000': {'enb-1-CH-0==>ue-1-CH-0': 1.0},
 '2680.000000': {'ue-1-CH-0==>enb-1-CH-0': 1.0, 'ue-1-CH-0==>enb-1-CH-1': 1.0}}

# Modifying the Channel & Examples

In this part, we will give details about how to configure CHEM for different scenarios. The below lines provide examples. We will start with changing channel coefficients. To change the specific individual channel, ``` change_channel_coeffs(coeff, freq, individual-channel-id)``` function needs to be used. As for now, user can only set 1 parameter. The example given below, change the available 2 channels coefficients to 0.1. This coefficient is multiplied with the received signal in the CHEM. Simply, user will be changing the power level of the signal with this function

In [7]:
ch.change_channel_coeffs(1., 2680, 'ue-1-CH-0==>enb-1-CH-0')

{'result': False}


True

In [8]:
ch.change_channel_coeffs(1., 2560, 'enb-1-CH-0==>ue-1-CH-0')

{'result': False}


True

To change the antenna model for specific node, ``` changeAntenna(node_name, antenna_model)``` function needs to be used. As of now, there are 2 supported antenna models which are __ISOTROPIC__ and __OMNI_DIRECTIONAL__ antennas. As default, CHEM starts the nodes with __ISOTROPIC__ antenna model. Therefore, any changes need to be done everytime CHEM starts. As a note, this function also returns a boolean if the operation is successful or not. 

In [9]:
ch.changeAntenna("enb-1", "OMNI_DIRECTIONAL") ## or "ISOTROPIC"

{'result': False}


True

By default, CHEM connects to vehicle MAVLINKs as configured inside the CHEM configuration file(```~/.config/CHEM/config.json```). Also, the path loss model is __FREE_SPACE__ by default. This option apply path loss while the vehicle is moving. AERPAW users don't need to do any configuration related to vehicles since it's going to be configured before it's given to user. To change the path loss, channel information(__not__ the individual) needs to used. The function for changing pathLoss mode is ```changePathLoss(channel-id(freq in MHz), pathloss_mode, groundReflectionCoeff)```. There are 3 path loss models,

* __NONE__ (No path loss is applied and signal doesn't vary in terms of path loss)
* __FREE_SPACE__
* __2_RAY__

As a note, this function also returns a boolean if the operation is successful or not. 


For groundReflectionCoeff, this is specific to __2_RAY__ path loss model. 

* __NONE__ or __FREE_SPACE__ : These models don't require any parameters. It can be left empty.
* __2_RAY__: The groundReflectionCoeff is -1 by default. This parameter needs to be set when the path loss model is changed. 
    

In [10]:
ch.changePathloss(2560, "NONE")

True

It's also possible to add noise to channel. As of now, CHEM only has __AWGN__ as an option. By default, CHEM starts without any noise. User specifically choose it for __individual_channels__. To change the noise, ```changeNoiseModel(noiseModel, noisePower(in dB), freq(in MHz), individual-channel-id)```. The supported options for noiseModel are,

* __NONE__
* __AWGN__

As a note, this function also returns a boolean if the operation is successful or not. This function is being __developed__ right now, it might have some problems.

In [11]:
ch.changeNoiseModel("AWGN", 20, 2560, 'enb-1-CH-0==>ue-1-CH-0')

{'result': False}
